In [15]:
from neuralnetwork import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
data = pd.read_csv('breast_cancer.csv')
data

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
678,3,1,1,1,3,2,1,1,1,2
679,2,1,1,1,2,1,1,1,1,2
680,5,10,10,3,7,3,8,10,2,4
681,4,8,6,4,3,4,10,6,1,4


In [17]:
data['Class'] = data['Class'] // 4
data['Class'].unique()

array([0, 1], dtype=int64)

In [18]:
x = data.drop(columns=['Class'])
y = data['Class']
print('x =\n', x)
print('\ny =', y)

x =
      Clump Thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
0                  5                        1                         1   
1                  5                        4                         4   
2                  3                        1                         1   
3                  6                        8                         8   
4                  4                        1                         1   
..               ...                      ...                       ...   
678                3                        1                         1   
679                2                        1                         1   
680                5                       10                        10   
681                4                        8                         6   
682                4                        8                         8   

     Marginal Adhesion  Single Epithelial Cell Size  Bare Nuclei  \
0                    1    

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train, y_train

(     Clump Thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
 426                5                        2                         2   
 213                6                        1                         3   
 578                5                        1                         2   
 186                5                        1                         1   
 636                5                        1                         1   
 ..               ...                      ...                       ...   
 351                2                        1                         1   
 453                4                        1                         1   
 560                5                        1                         2   
 309                5                        4                         6   
 151                5                        5                         5   
 
      Marginal Adhesion  Single Epithelial Cell Size  Bare Nuclei  \
 426             

In [20]:
model = DeepNeuralNetwork(Layer(9),
                          Layer(18),
                          Layer(1, activation_func='sigmoid', train_bias=False))

### Weights and bias init

In [21]:
for i in range(model.layers_len-1):
    print(f'weight init layers {i}= :\n', model.weights[i])
    print(f'\nshape of weights = {model.weights[i].shape}')
    print(f'\nbias init layers {i}= \n', model.layers[i].biases)
    print(f'\nshape of bias = {model.layers[i].biases.shape}')
    

weight init layers 0= :
 [[0.03850611 0.16833579 0.26147523 0.17116274 0.08276913 0.09074198
  0.19476569 0.15604308 0.21399123 0.18773366 0.17392643 0.16380042
  0.05664675 0.22689684 0.15777429 0.06198884 0.26696031 0.00801413]
 [0.12668397 0.19298902 0.15420456 0.21327437 0.24614504 0.23565387
  0.16205516 0.02745735 0.2009539  0.21923909 0.00792125 0.10438696
  0.09820167 0.09001921 0.01205923 0.26477808 0.07290515 0.21802659]
 [0.19643787 0.13679813 0.25808049 0.24099646 0.15647753 0.09859708
  0.07927805 0.25837889 0.2548802  0.23565643 0.23677502 0.17569356
  0.25921905 0.26086135 0.20295093 0.01430997 0.22151582 0.22039897]
 [0.06632812 0.04478653 0.23173923 0.19415225 0.22200154 0.04368559
  0.03490455 0.16664434 0.19274129 0.08713557 0.23110248 0.20479902
  0.18833746 0.08229376 0.03413032 0.01934502 0.23008272 0.22988597]
 [0.11242983 0.21051483 0.21086974 0.24522104 0.0647822  0.03823313
  0.14060891 0.03133676 0.25873232 0.12642081 0.18219295 0.19098226
  0.09967657 0.1679

### Training

In [22]:
def fit(x_train, y_train, lr = 0.01, batch_size = 32, epochs = 1000):
    N, d = x_train.shape
    for it in range(epochs):
        rand_id = np.random.choice(N, size=batch_size, replace=False)
        for i in rand_id:
            xi = np.array([x_train.iloc[i, :]]).T
            yi = y_train.iloc[i]
            activation_cache, linear_cache = model.forward(xi)
            loss = model.calculate_loss(yi, activation_cache[-1])
            error = model.error(loss, linear_cache)
            # model.backpropagation(error, activation_cache)
            model.backpropagation(linear_cache, activation_cache, yi)
            model.update(lr = lr)

In [23]:
fit(x_train, y_train, lr=0.001, batch_size=32, epochs=1000)

In [24]:
y_pred = model.predict(x_train)
print(y_pred)
count_0 = 0
count_1 = 0
for i in y_pred:
    if i == 0:
        count_0 += 1
    else:
        count_1 += 1
print('number of 1 :', count_1)
print('number of 0 :', count_0)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 

### Accuracy

In [25]:
acc = model.accuracy(y_train, y_pred)
print("accuracy train:", acc)

accuracy train: 0.967032967032967


In [26]:
y_pred_test = model.predict(x_test)
acc = model.accuracy(y_test, y_pred_test)
print("accuracy test:", acc)

accuracy test: 0.9635036496350365


### Loss

In [27]:
loss_train = model.calculate_loss(y_train, y_pred)
print("loss train :", loss_train)

loss train : 1.1386483023727907


In [28]:
loss_test = model.calculate_loss(y_test, y_pred_test)
print("loss test :", loss_test)

loss test : 1.2605509574409777
